In [1]:
import numpy as np
import pandas as pd
import json
import h5py

from obspy import Stream, Trace, UTCDateTime

from tqdm import tqdm
from tsfresh import select_features, extract_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import EfficientFCParameters
from tsfresh.feature_extraction.settings import from_columns

%matplotlib inline

ImportError: No module named 'tsfresh'

In [ ]:
# Folders:
root = 'train_test_mixed'
train_folder = f'{root}/train/'
test_folder = f'{root}/test/'
mixed_folder = f'{root}/mixed/'

earthquake_samples = 'earthquake_sample_events.h5'
explosion_samples = 'explosion_sample_events.h5'

In [ ]:
# Mixed, train, test sets:
mixed_info = np.load(mixed_folder + 'mixed_info.npy', allow_pickle=True)
mixed_trace = np.load(mixed_folder + 'mixed_trace.npy', allow_pickle=True)

train_info = np.load(train_folder + 'train_info.npy', allow_pickle=True)
train_trace = np.load(train_folder + 'train_trace.npy', allow_pickle=True)

test_info = np.load(test_folder + 'test_info.npy', allow_pickle=True)
test_trace = np.load(test_folder + 'test_trace.npy', allow_pickle=True)

In [ ]:
def load_datasets(dataset):
    with h5py.File(dataset, 'r') as fin:
        event_info_dataset = fin.get('event_info')
        trace_dataset = fin.get('traces')

        # Convert to numpy arrays
        event_info = np.array(event_info_dataset)
        trace_data = np.array(trace_dataset)
    event_type = dataset.split('_')[0]

    print(f'Loaded {event_info.shape[0]} {event_type} events')
    return event_info, trace_data

In [2]:
earthquake_info, earthquake_trace = load_datasets(earthquake_samples)
explosion_info, explosion_trace = load_datasets(explosion_samples)

NameError: name 'load_datasets' is not defined

In [22]:
def create_depth_list(info):
    depths = np.empty((len(info)))
    j = 0
    for i in info:
        i = json.loads(i)
        depths[j] = float(i['origins'][0]['depth'])
        j += 1
    return depths
    

In [35]:
earth_depths = create_depth_list(earthquake_info)
explo_depths = create_depth_list(explosion_info)

In [36]:
explo_depths[5]

0.0

In [37]:
pd.DataFrame(explo_depths).describe()

,0
count,198.0
mean,0.0
std,0.0
min,0.0
25%,0.0
50%,0.0
75%,0.0
max,0.0


In [38]:
pd.DataFrame(earth_depths).describe()

,0
count,152.000000
mean,8.284255
std,5.510407
min,0.000000
25%,5.000000
50%,7.600000
75%,10.000000
max,37.400000


In [41]:
def create_mags_list(info):
    mags = np.empty((len(info)))
    j = 0
    for i in info:
        i = json.loads(i)
        mags[j] = float(i['magnitudes'][0]['mag'])
        j += 1
    return mags

In [42]:
earth_mags = create_mags_list(earthquake_info)
explo_mags = create_mags_list(explosion_info)

In [43]:
pd.DataFrame(earth_mags).describe()

,0
count,152.000000
mean,1.414408
std,0.566045
min,0.600000
25%,1.000000
50%,1.400000
75%,1.700000
max,4.484000


In [44]:
pd.DataFrame(explo_mags).describe()

,0
count,198.000000
mean,1.439394
std,0.150004
min,1.300000
25%,1.300000
50%,1.400000
75%,1.500000
max,1.900000
